In [78]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D
from keras.models import Model
from keras import backend as K

n_features = 4
n_runs = 300
seq_length = [180, 360, 540, 720, 900]

def read_sequences():
    run_list_mix = []
    run_list_ordered = []
    for index in range(n_runs):
        run_csv = pd.read_csv("Mix_sequences_var_length/run^"+str(index))
        run_csv = run_csv.drop(columns=['Unnamed: 0'])
        run_list_mix.append(run_csv)
    for index, length in enumerate(seq_length):
        run_list_ordered.append([])
        for run in run_list_mix: 
            if len(run) == length:
                run_list_ordered[index].append(run)    
    return run_list_ordered
    
    
def stadard_sequences(sequences):
    for index, seqs in enumerate(sequences):
        for i, seq in enumerate(seqs):
            seqs[i] = StandardScaler().fit_transform(seq)
    return sequences                    


trainX_sequences = stadard_sequences(read_sequences())
for i, leng in enumerate(seq_length):
    trainX_sequences[i] = np.reshape(trainX_sequences[i], (-1, leng, n_features))


def train_generator():
    index = 0
    while True:
        x_train = trainX_sequences[index]
        index += 1
        yield (x_train,x_train) 


ValueError: None values not supported.

In [86]:
from keras.layers import Conv2DTranspose
from keras.losses import mse

intermediate_dimensions = 50
latent_dim = 20
filters = 50


def Conv1DTranspose(input_tensor, filters, kernel_size,last, strides=2, padding='same'):
        if last:
            activation = 'linear'
        else:
            activation = 'relu'
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), 
                            strides=(strides, 1), padding=padding,
                            activation=activation)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x


def sampling(args):
        z_mean, z_log_var = args
        batch = K.shape(z_mean)[0]
        dim = K.int_shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon

def create_vae(verbose):
    inputs = Input(shape=(None, n_features))
    x = inputs
    
    for i in range(2):
        x = Conv1D(filters=filters,
                   kernel_size=2,
                   strides=2,
                   padding='same')(x)
    
    shape = K.int_shape(x)
    #x = Flatten()(x)
    # x = Conv1D(filters=50,
    #            kernel_size=2,
    #            strides=2,
    #            padding='same')(x)
   
    #x = Reshape((shape[1]*shape[2],))(x)
    embeddings = Dense(latent_dim)(x)
    
    # z_mean = Dense(latent_dim, name='z_mean',)(embeddings)
    # z_log_var = Dense(latent_dim, name='z_log_var')(embeddings)
    # z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
    
    encoder = Model(inputs, embeddings, name='encoder')
    if verbose:
        encoder.summary()
    
    latent_inputs = Input(shape=(latent_dim,), name='latent_inputs')
    x = Dense(shape[1]*shape[2])(latent_inputs)
    x = Reshape((shape[1], shape[2]))(x)
    
    for i in range(2):
        x = Conv1DTranspose(input_tensor=x,
                            filters=filters,
                            kernel_size=2,
                            last=False,
                            padding='same')
    
    
    outputs = Conv1DTranspose(input_tensor=x,
                              filters=n_features,
                              kernel_size=2,
                              strides=1,
                              last=True,
                              padding='same')
    
    decoder = Model(latent_inputs, outputs)
    if verbose:
        decoder.summary()
    
    outputs = decoder(encoder.outputs[2])
    reconstruction_loss = mse(K.flatten(inputs), K.flatten(outputs))
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    loss = K.mean(reconstruction_loss)
    
    
    vae = Model(inputs, outputs, name='vae')
    vae.add_loss(loss)
    
    vae.compile(optimizer='rmsprop')
    
    return (vae, encoder,decoder)


vae, enc, dec = create_vae(verbose=True)

for i in range(len(seq_length)):
    vae.fit(trainX_sequences[i],epochs=10, verbose=1)


ValueError: None values not supported.